# FewShotPrompt

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os

os.environ["LANGSMITH_PROJECT"]

'LANGCHAIN-BASIC'

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1,
    model='gpt-4.1-mini',
    verbose=True
)

In [3]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

examples = [
    {'question' : '안녕하세요', # 표준어
        'answer' : '안녕하나'}, # 경상도 사투리
    {'question' : '맛있게 먹어',
        'answer' : '마이 무라'},
    {'question' : '뭐하니',
        'answer' : '뭐하노'},
    {'question' : '뭐라고',
        'answer' : '뭐라카노'},
    {'question' : '밥 먹었어?',
        'answer' : '밥 뭇나?'},
]

In [4]:
example_prompt = PromptTemplate.from_template(
"""
[질문 내용]    
{question} 을/를 경상도 사투리로 어떻게 말하나요?

[경상도 사투리]
{answer}
"""
)

example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n[질문 내용]    \n{question} 을/를 경상도 사투리로 어떻게 말하나요?\n\n[경상도 사투리]\n{answer}\n')

In [5]:
# prefix + (example_prompt(format) * examples(data)) + suffix(필수요소 - 적어도 ""으로 처리 필요)
prompt_template = FewShotPromptTemplate(
    examples = examples, # 학습용 예시 data
    example_prompt = example_prompt, # examples를 어떤 식으로 변경할지 알려주는 format
    prefix = """다음은 표준어 문장을 경상도 사투리로 바꾼 예시입니다. 반드시 같은 형식을 따라 답하세요.\n\n""", # 응답에 대한 가이드
    suffix= # 실제 질문이 들어갈 프롬프트
    """질문 내용 : {question}\n 사투리 :""",
    input_variables=["question"]
)

In [6]:
question = "안녕하세요 지금 뭐하고 있어요?"
q_prompt = prompt_template.format(question=question)
print(q_prompt)

다음은 표준어 문장을 경상도 사투리로 바꾼 예시입니다. 반드시 같은 형식을 따라 답하세요.




[질문 내용]    
안녕하세요 을/를 경상도 사투리로 어떻게 말하나요?

[경상도 사투리]
안녕하나



[질문 내용]    
맛있게 먹어 을/를 경상도 사투리로 어떻게 말하나요?

[경상도 사투리]
마이 무라



[질문 내용]    
뭐하니 을/를 경상도 사투리로 어떻게 말하나요?

[경상도 사투리]
뭐하노



[질문 내용]    
뭐라고 을/를 경상도 사투리로 어떻게 말하나요?

[경상도 사투리]
뭐라카노



[질문 내용]    
밥 먹었어? 을/를 경상도 사투리로 어떻게 말하나요?

[경상도 사투리]
밥 뭇나?


질문 내용 : 안녕하세요 지금 뭐하고 있어요?
 사투리 :


In [7]:
question = '안녕하세요 지금 뭐하고 있어요?'

chain = prompt_template | model

response = chain.invoke({'question' : question})
print(response.content)

[경상도 사투리]  
안녕하나 지금 뭐하노?
